In [1]:
# Initial imports
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [10]:
# Reading the News API key enviroment variable and xreate a newsapi client
api_key = os.getenv("news_api")
newsapi = NewsApiClient(api_key=api_key)

In [26]:
# Fetch the stock news
stock_news = newsapi.get_everything(
    q="FB",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

#stock_news

In [27]:
print(f"Total articles: {stock_news['totalResults']}")
stock_news["articles"][0]

Total articles: 1560


{'source': {'id': None, 'name': 'StockNews.com'},
 'author': 'Andrew Hecht',
 'title': 'Apple and Facebook Knock the Cover Off the Ball in Q1 Earnings',
 'description': 'As most companies have reported earnings for the first quarter, two stand out with massive growth, Apple (AAPL) and Facebook (FB). With Facebook facing changes at Apple, can both companies keep this growth up? Read more to find out.',
 'url': 'https://stocknews.com/news/aapl-fb-apple-and-facebook-knock-the-cover-off-the-ball-in/',
 'urlToImage': 'https://assets.entrepreneur.com/providers/stocknews/hero-image-stocknews-371621.jpeg',
 'publishedAt': '2021-05-11T13:52:35Z',
 'content': 'This story originally appeared on StockNewsAs most companies have reported earnings for the first quarter, two stand out with massive growth, Apple (AAPL) and Facebook (FB). With Facebook facing chan… [+7604 chars]'}

In [28]:
analyzer.polarity_scores('good evil')

{'neg': 0.603, 'neu': 0.0, 'pos': 0.397, 'compound': -0.3612}

In [29]:
# Create the sentiment scores DataFrame
stock_sentiments = []

for article in stock_news["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        stock_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
        })
        
    except AttributeError:
        pass
    
stock_df = pd.DataFrame(stock_sentiments)
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
stock_df = stock_df[cols]
stock_df.head()

,date,text,compound,positive,negative,neutral
0,2021-05-11,This story originally appeared on StockNewsAs ...,0.3818,0.077,0.0,0.923
1,2021-04-27,When a new medium shows up that allocates atte...,0.6239,0.110,0.0,0.890
2,2021-05-10,This story originally appeared on StockNewsThr...,0.0000,0.000,0.0,1.000
3,2021-05-13,"shout out to four year old me, who on hearing ...",0.4215,0.078,0.0,0.922
4,2021-05-03,The Facebook Oversight Board said it will anno...,0.0000,0.000,0.0,1.000


In [15]:
stock_df['text'][1]

'When a new medium shows up that allocates attention, someone comes out ahead.\r\nThey win a Google search, amass a ton of FB followers, hit the jackpot on Twitter or even Clubhouse.\r\n“They picked me!” … [+771 chars]'

In [16]:
# Get descriptive stats from the DataFrame
stock_df.describe()

,compound,positive,negative,neutral
count,100.000000,100.00000,100.000000,100.000000
mean,0.095278,0.06277,0.038870,0.898350
std,0.432702,0.07017,0.055193,0.079456
min,-0.802000,0.00000,0.000000,0.654000
25%,-0.146575,0.00000,0.000000,0.848000
50%,0.000000,0.05300,0.000000,0.907000
75%,0.442725,0.09525,0.074500,0.951750
max,0.889300,0.30500,0.228000,1.000000
